**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [3]:
# Import the data set
df=pd.read_csv('D:\Technocolab\Mini Project 2\Task 1\cleaned_data.csv')

In [4]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [6]:
# Define the sigmoid function
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

In [7]:
sigmoid(0)

0.5

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [8]:
# Create a train/test split
features = ["PAY_1","LIMIT_BAL"]
X = df[features].values
y = df["default payment next month"].values

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2 ,random_state = 24)

In [10]:
print("X_train shape is :",X_train.shape)
print("X_test shape is :",X_test.shape)
print("y_train shape is :",y_train.shape)
print("X_test shape is :",y_test.shape)

X_train shape is : (21331, 2)
X_test shape is : (5333, 2)
y_train shape is : (21331,)
X_test shape is : (5333,)


______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [11]:
from sklearn.linear_model import LogisticRegression
Logistic_Regression= LogisticRegression(solver="liblinear")

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [12]:
# Fit the logistic regression model on training data
Logistic_Regression.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [13]:
# Make predictions using `.predict()`
y_pred = Logistic_Regression.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [14]:
# Find class probabilities using `.predict_proba()`
pred_proba = Logistic_Regression.predict_proba(X_test)
pred_proba

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [15]:
# Add column of 1s to features
ones_feats = np.hstack([np.ones((X_test.shape[0],1)),X_test])

In [16]:
# Get coefficients and intercepts from trained model
coeff_interc_feats=np.concatenate([Logistic_Regression.intercept_.reshape(1,1), Logistic_Regression.coef_],axis=1)
coeff_interc_feats

array([[-6.57647457e-11,  8.27451187e-11, -6.80876727e-06]])

In [17]:
# Manually calculate predicted probabilities
pred_proba_manual=np.dot(coeff_interc_feats,np.transpose(ones_feats))
pred_proba_manual

array([[-1.08940276, -0.34043836, -1.36175345, ..., -0.34043836,
        -1.56601647, -0.68087673]])

In [18]:
sig_man_predproba = sigmoid(pred_proba_manual)
sig_man_predproba

array([[0.25173076, 0.415703  , 0.20395547, ..., 0.415703  , 0.17278502,
        0.33606565]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [19]:
# Manually calculate predicted classes
pred_manual= sig_man_predproba >= 0.5
pred_manual

array([[False, False, False, ..., False, False, False]])

In [22]:
# Compare to scikit-learn's predicted classes
np.array_equal(pred_manual,y_pred.reshape(1,-1))

True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [23]:
from sklearn.metrics import roc_auc_score

In [24]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
roc_auc_score(y_test,pred_proba[:,1])

0.627207450280691

In [25]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test,pred_proba_manual.reshape(pred_proba_manual.shape[1],))

0.627207450280691